In [ ]:
# To upload our datasets from our working directory we need to mount our drive contents to the colab environment.
# For the code to do so you can search “mount” in code snippets or use the code given below.
# Our entire drive contents are now mounted on colab at the location “/gdrive”.

from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive


Mounted at /gdrive
/gdrive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
pd.set_option('display.max_columns',None)#displaying long list of columns
pd.set_option('display.max_rows', None)#displaying long list of rows
pd.set_option('display.width', 1000)#width of window


In [ ]:
trainfile = r'/gdrive/My Drive/train.csv'
trainData = pd.read_csv(trainfile)  #creates a dataframe
testfile = r'/gdrive/My Drive/test.csv'
testData = pd.read_csv(testfile)  #creates a dataframe
print(trainData.shape)
print(testData.shape)

(137, 43)
(100000, 42)


In [ ]:
#Extract Target Column before doing missing value substitutions and one-hot encoding======
Target_Train_Cols = trainData["revenue"]#make copy of target column
trainData = trainData.drop(["revenue"], axis=1) #extracting training data without the target column

print(trainData.shape)

(137, 42)


In [ ]:
#DROP COLUMNS WITH LOTS OF MISSING VALUES===============================
#CAN ALSO DROP ROWS WITH LOTS OF MISSING VALUES
#Combine Train data and test data first so that the SAME COLUMNS are DROPPED in each
combined_Data = pd.concat([trainData, testData], keys=[0,1])

combined_Data.isnull().sum().sort_values(ascending=False)


Id            0
P27           0
P19           0
P20           0
P21           0
P22           0
P23           0
P24           0
P25           0
P26           0
P28           0
Open Date     0
P29           0
P30           0
P31           0
P32           0
P33           0
P34           0
P35           0
P36           0
P18           0
P17           0
P16           0
P15           0
City          0
City Group    0
Type          0
P1            0
P2            0
P3            0
P4            0
P5            0
P6            0
P7            0
P8            0
P9            0
P10           0
P11           0
P12           0
P13           0
P14           0
P37           0
dtype: int64

In [ ]:
#Define threshold for dropping columns
percent=int(0.6*(combined_Data.shape[0]))
print(percent)
#Drop columns that have less than "thresh" number of non_Nans
td1=combined_Data.dropna(thresh=percent,axis=1)
print(td1.shape)

60082
(100137, 42)


In [ ]:
#look at what other columns have missing values
td1.isnull().sum().sort_values(ascending=False)

Id            0
P27           0
P19           0
P20           0
P21           0
P22           0
P23           0
P24           0
P25           0
P26           0
P28           0
Open Date     0
P29           0
P30           0
P31           0
P32           0
P33           0
P34           0
P35           0
P36           0
P18           0
P17           0
P16           0
P15           0
City          0
City Group    0
Type          0
P1            0
P2            0
P3            0
P4            0
P5            0
P6            0
P7            0
P8            0
P9            0
P10           0
P11           0
P12           0
P13           0
P14           0
P37           0
dtype: int64

In [ ]:
#NOW IMPUTE MISSING VALUES FOR THE OTHER COLUMNS=========================
#IMPUTE (SUBSTITUTE) MEAN VALUES FOR NaN IN NUMERIC COLUMNS
numeric=td1.select_dtypes(include=['int','float64']).columns
for num in numeric:
  td1[num]=td1[num].fillna(td1[num].mean())

#IMPUTE (SUBSTITUTE) MODE VALUES FOR NaN IN CATEGORICAL COLUMNS
train_cat_cols = td1.select_dtypes(exclude=['int','float64']).columns#selecting the categorical columns
for colss in train_cat_cols:
  if(td1.iloc[0][colss]=="N"):
        td1[colss]=td1[colss].fillna("N")
  else:
    td1[colss]=td1[colss].fillna(td1[colss].mode())

print(td1.head(20))



      Id   Open Date            City  City Group Type  P1   P2   P3   P4  P5  P6  P7  P8  P9  P10  P11  P12  P13  P14  P15  P16  P17  P18  P19  P20  P21  P22  P23  P24  P25   P26  P27   P28  P29  P30  P31  P32  P33  P34  P35  P36  P37
0 0    0  07/17/1999        İstanbul  Big Cities   IL   4  5.0  4.0  4.0   2   2   5   4   5    5    3    5  5.0    1    2    2    2    4    5    4    1    3    3    1    1   1.0  4.0   2.0  3.0    5    3    4    5    5    4    3    4
  1    1  02/14/2008          Ankara  Big Cities   FC   4  5.0  4.0  4.0   1   2   5   5   5    5    1    5  5.0    0    0    0    0    0    3    2    1    3    2    0    0   0.0  0.0   3.0  3.0    0    0    0    0    0    0    0    0
  2    2  03/09/2013      Diyarbakır       Other   IL   2  4.0  2.0  5.0   2   3   5   5   5    5    2    5  5.0    0    0    0    0    0    1    1    1    1    1    0    0   0.0  0.0   1.0  3.0    0    0    0    0    0    0    0    0
  3    3  02/02/2012           Tokat       Other   IL   6  4

In [ ]:
#CHECK IF THERE ARE ANY REMAINING MISSING VALUES
td1.isnull().sum().sort_values(ascending=False)

Id            0
P27           0
P19           0
P20           0
P21           0
P22           0
P23           0
P24           0
P25           0
P26           0
P28           0
Open Date     0
P29           0
P30           0
P31           0
P32           0
P33           0
P34           0
P35           0
P36           0
P18           0
P17           0
P16           0
P15           0
City          0
City Group    0
Type          0
P1            0
P2            0
P3            0
P4            0
P5            0
P6            0
P7            0
P8            0
P9            0
P10           0
P11           0
P12           0
P13           0
P14           0
P37           0
dtype: int64

In [ ]:
#DROP COLUMNS THAT STILL HAVE NULL VALUES
print(td1.shape)
td1=td1.drop(columns=["PropertyField32", "PropertyField34"])
td1=td1.drop(columns=["PropertyField5"])
td1.isnull().sum()
print(td1.shape)

(434589, 296)
(434589, 293)


In [ ]:
#DO ONE-HOT ENCODING ON CATEGORICAL VARIABLES==============================================
#The below function returns a list of categorical features which are not numeric.
train_cat_cols = td1.select_dtypes(exclude=['float','int']).columns #selecting the categorical columns
print(train_cat_cols.shape)
print(train_cat_cols)

#If there are categorical columns which are encoded as numeric ones
#then we need to explicitly enter the column names in a list and concatenate the two lists in python.
#ONE-HOT ENCODING-generate one-hot encoding on a common basis -THIS TAKES 30 MINS

combined_Data = pd.get_dummies(td1,train_cat_cols)
combined_Data.head(10)


(4,)
Index(['Open Date', 'City', 'City Group', 'Type'], dtype='object')


Id  P1   P2   P3   P4  P5  P6  P7  P8  P9  P10  P11  P12  P13  P14  P15  P16  P17  P18  P19  P20  P21  P22  P23  P24  P25  P26  P27  P28  P29  P30  P31  P32  P33  P34  P35  P36  P37  Open Date_01/01/2000  Open Date_01/03/2013  Open Date_01/03/2014  Open Date_01/04/2014  Open Date_01/05/2000  Open Date_01/07/2000  Open Date_01/07/2011  Open Date_01/09/2010  Open Date_01/14/2011  Open Date_01/15/2013  Open Date_01/16/1999  Open Date_01/16/2009  Open Date_01/17/2009  Open Date_01/17/2012  Open Date_01/18/2011  Open Date_01/18/2013  Open Date_01/20/2012  Open Date_01/22/2007  Open Date_01/22/2011  Open Date_01/22/2013  Open Date_01/23/2009  Open Date_01/25/2010  Open Date_01/25/2014  Open Date_01/26/2009  Open Date_01/26/2012  Open Date_01/27/2005  Open Date_01/28/1998  Open Date_01/31/2012  Open Date_02/01/2011  Open Date_02/01/2012  Open Date_02/02/2012  Open Date_02/06/2007  Open Date_02/07/2007  Open Date_02/07/2009  Open Date_02/07/2012  Open Date_02/08/2007  \
0 0   0   4  5.0  4.0  4.0   2   2   5   4   5    5    3    5  5.0    1    2    2    2    4    5    4    1    3    3    1    1  1.0  4.0  2.0  3.0    5    3    4    5    5    4    3    4                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0   
  1   1   4  5.0  4.0  4.0   1   2   5   5   5    5    1    5  5.0    0    0    0    0    0    3    2    1    3    2    0    0  0.0  0.0  3.0  3.0    0    0    0    0    0    0    0    0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0   
  2   2   2  4.0  2.0  5.0   2   3   5   5   5    5    2    5  5.0    0    0    0    0    0    1    1    1    1    1    0    0  0.0  0.0  1.0  3.0    0    0    0    0    0    0    0    0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0                     0   
  3   3   6  4.5  6.0  6.0   4   4  10   8  10   10    8   10  7.5    6    4 

In [ ]:
#Separate Train data and test data
trainData = td1.xs(0)
testData = td1.xs(1)
print(trainData.shape)
print(testData.shape)

trainData=pd.concat([trainData,Target_Train_Cols], axis=1)
print(trainData.shape)


(137, 42)
(100000, 42)
(137, 43)


In [ ]:
export_csv = trainData.to_csv(r'/gdrive/My Drive/Preprocess_Train_Assignment4.csv')
exporttest_csv = testData.to_csv(r'/gdrive/My Drive/Preprocess_Test_Assignment4.csv')